In [1]:
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import plotly.express as px
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from PIL import Image
import io
import base64
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix
import gc

from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAvgPool2D,Conv2D, MaxPool2D, BatchNormalization, Dropout, Input, LeakyReLU, Reshape, Conv2DTranspose, Concatenate, Lambda, MaxPooling2D
from tensorflow.keras.initializers import GlorotUniform
 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model


In [2]:
# Kaggle API Token
from google.colab import files
files.upload()

# Setup Kaggle API
!pip install kaggle --quiet
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the Dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
 # Extract the Dataset
!unzip -q chest-xray-pneumonia.zip

Saving kaggle.json to kaggle.json
100% 2.29G/2.29G [01:46<00:00, 23.5MB/s]
100% 2.29G/2.29G [01:46<00:00, 23.2MB/s]


In [ ]:
# Set up the directory path for training and testing images
base_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray'

In [ ]:
import os
import shutil

# Function to copy files from source directories to a target directory
def copy_files(source_dir, target_dir, num_files=None):
   count = 0
   for filename in os.listdir(source_dir):
      if num_files is not None and count >= num_files:
         break
      file_path = os.path.join(source_dir, filename)
      if os.path.isfile(file_path):
         count += 1
         shutil.copy(file_path, target_dir)

   print(f'Copied {count} files from {source_dir} to {target_dir}')

# Set up the directory path for training, validation, and testing images
base_dir = './chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'test')
test_dir = os.path.join(base_dir, 'val')


# Create a "train_undersampled" directory to store the undersampled training images
train_undersampled = os.path.join(base_dir, 'train_undersampled')
os.makedirs(train_undersampled, exist_ok=True)
# Create new 'NORMAL' and 'PNEUMONIA' directories inside the train_undersampled directory
normal_dir_undersampled = os.path.join(train_undersampled, 'NORMAL')
pneumonia_dir_undersampled = os.path.join(train_undersampled, 'PNEUMONIA')
os.makedirs(normal_dir_undersampled, exist_ok=True)
os.makedirs(pneumonia_dir_undersampled, exist_ok=True)
# Copy images from the training 'NORMAL' and 'PNEUMONIA' directories to the 'train_undersampled' directory in a 1:1 ratio
# Calculate the number of files in the 'NORMAL' and 'PNEUMONIA' directories
normal_count = len(os.listdir(f'{train_dir}/NORMAL'))
pneumonia_count = len(os.listdir(f'{train_dir}/PNEUMONIA'))
minority_count = min(normal_count, pneumonia_count)
# Copy minority_count number of images from 'train_dir' to 'train_undersampled'
copy_files(os.path.join(train_dir, 'NORMAL'), normal_dir_undersampled, minority_count)
copy_files(os.path.join(train_dir, 'PNEUMONIA'), pneumonia_dir_undersampled, minority_count)


# Create a 'val_undersampled' directory to store the undersampled validation images
val_undersampled = os.path.join(base_dir, 'val_undersampled')
os.makedirs(val_undersampled, exist_ok=True)
# Create new 'NORMAL' and 'PNEUMONIA' directories inside the val_undersampled directory
normal_dir_undersampled = os.path.join(val_undersampled, 'NORMAL')
pneumonia_dir_undersampled = os.path.join(val_undersampled, 'PNEUMONIA')
os.makedirs(normal_dir_undersampled, exist_ok=True)
os.makedirs(pneumonia_dir_undersampled, exist_ok=True)
# Copy images from the validation 'NORMAL' and 'PNEUMONIA' directories to the 'val_undersampled' directory in a 1:1 ratio
# Calculate the number of files in the 'NORMAL' and 'PNEUMONIA' directories
normal_count = len(os.listdir(f'{val_dir}/NORMAL'))
pneumonia_count = len(os.listdir(f'{val_dir}/PNEUMONIA'))
minority_count = min(normal_count, pneumonia_count)
# Copy minority_count number of images from 'val_dir' to 'val_undersampled'
copy_files(os.path.join(val_dir, 'NORMAL'), normal_dir_undersampled, minority_count)
copy_files(os.path.join(val_dir, 'PNEUMONIA'), pneumonia_dir_undersampled, minority_count)

In [4]:

# Preprocessing the images - setting up image data generators
batch_size = 24
image_size = 224

# Limited data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                           width_shift_range=0.2, height_shift_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
combined_datagen = ImageDataGenerator(rescale=1./255)

balanced_train_generator = train_datagen.flow_from_directory(
      train_undersampled,
      target_size=(image_size, image_size),
      batch_size=batch_size,
      class_mode='binary',
      shuffle=True
)

balanced_val_generator = val_datagen.flow_from_directory(
      val_undersampled,
      target_size=(image_size, image_size),
      batch_size=batch_size,
      class_mode='binary',
      shuffle=True
)

test_generator = test_datagen.flow_from_directory(
      test_dir,
      target_size=(image_size, image_size),
      batch_size=batch_size,
      class_mode='binary',
      shuffle=False
)

# Use tf.data.Dataset for efficient data loading
def generator_wrapper(generator):
   for x_batch, _ in generator:
      x_batch = tf.cast(x_batch, tf.float32)
      yield (x_batch, x_batch)
# Shuffle buffer size
buffer_size = 2

train_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(balanced_train_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(balanced_val_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
   lambda: generator_wrapper(test_generator),
   output_signature=(
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32),
      tf.TensorSpec(shape=(None, image_size, image_size, 3), dtype=tf.float32))
).prefetch(tf.data.AUTOTUNE)


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


# Convolutional Autoencoder

In [ ]:

# Encoder
conv_encoder = Sequential([
         Reshape([image_size, image_size, 3], name="input"), # input: 224x224x1
         BatchNormalization(),
         Conv2D(16, 3, padding="same", activation="relu"), 
         MaxPool2D(pool_size=2), # output: 112x112x16
         Dropout(0.15),
         BatchNormalization(),
         Conv2D(32, 3, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 56x56x32
         Dropout(0.35),
         BatchNormalization(),
         Conv2D(64, 3, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 28x28x64
         Dropout(0.55),
         BatchNormalization(),
         Conv2D(128, 5, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 14x14x128
         Dropout(0.35),
         BatchNormalization(),
         Conv2D(256, 5, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 7x7x256
         Dropout(0.15),
         BatchNormalization(),
         Conv2D(512, 5, padding="same", activation="relu"),
         MaxPool2D(pool_size=2), # output: 3x3x512
         Dropout(0.15),
         BatchNormalization(),
         Conv2D(90, 5, padding="same", activation="relu"),
         GlobalAvgPool2D(), # output: 1x1x90
      ])

# Decoder
conv_decoder = Sequential([
         Dense(28 * 28 * 512),
         Reshape((28, 28, 512), name="decoder_input"), # input: 28x28x512
         Conv2DTranspose(512, 5, strides=2, padding="same", activation="relu"), # output: 56x56x512
         Dropout(0.15),
         BatchNormalization(),
         Conv2DTranspose(256, 5, strides=2, padding="same", activation="relu"), # output: 112x112x256
         Dropout(0.25),
         BatchNormalization(),
         Conv2DTranspose(128, 5, strides=2, padding="same", activation="relu"), # output: 224x224x128
         Dropout(0.5),
         BatchNormalization(),
         Conv2DTranspose(64, 3, strides=1, padding="same", activation="relu"), # output: 224x224x64
         Dropout(0.25),
         BatchNormalization(),
         Conv2DTranspose(32, 3, strides=1, padding="same", activation="relu"), # output: 224x224x32
         Dropout(0.15),
         BatchNormalization(),
         Conv2DTranspose(16, 3, strides=1, padding="same", activation="relu"), # output: 224x224x16
         Dropout(0.05),
         BatchNormalization(),
         Conv2DTranspose(3, 3, strides=1, padding="same", activation="sigmoid"), # output: 224x224x1
         Reshape([image_size, image_size, 3], name="output")
      ])

# Autoencoder
conv_ae = Sequential([conv_encoder, conv_decoder])


conv_ae.compile(
    loss="mse", 
    # Use nadam with a 0.01 learning rate
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.05),
     metrics=[tf.keras.metrics.MeanSquaredError(),'accuracy'])

conv_ae.build(input_shape=(None, image_size, image_size, 3))
conv_ae.summary()



The MNIST AE example from the Hands-On Machine Learning text reduced the input rank from 28x28x1 = 784 to 30, a 96.17% reduction. We are attempting to reduce our input rank from 224x224x3 = 150,528 to around 100, a 99.93% reduction. To make that difference more apparent, if we were to reduce rank by 96.17% on our input, we would have a bottleneck tensor rank of 5765 (150,528 * 0.0383). This is a significant difference in the amount of information we are trying to compress.

In [ ]:
# Define the callbacks
ae_early_stopping = EarlyStopping(monitor='mean_squared_error', patience=5, verbose=1, mode='min', restore_best_weights=True)
ae_model_checkpoint = ModelCheckpoint('conv_ae.keras', monitor='mean_squared_error', verbose=1,mode='min', save_best_only=True)
ae_reduce_lr = ReduceLROnPlateau(monitor='mean_squared_error', factor=0.01, patience=4, verbose=1, mode='min')

# Define the callbacks for the classifier (monitor validation accuracy)
classifier_early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)
classifier_model_checkpoint = ModelCheckpoint('enhanced_clf_v1.keras', monitor='val_accuracy', verbose=1, mode='max', save_best_only=True)
classifier_reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.01, patience=4, verbose=1, mode='max')

In [ ]:

history = conv_ae.fit(
   train_dataset,
   epochs=30,
   #validation_data=test_dataset,
   steps_per_epoch=len(balanced_train_generator),
   #validation_steps=len(test_generator),
   callbacks=[
      ae_early_stopping,
      ae_model_checkpoint,
      ae_reduce_lr
   ])

In [ ]:
# Plot the training history using plotly

def plot_history(history):
   # Plot the mean squared error and accuracy
   fig = px.line(
      history.history,
      y=['mean_squared_error', 'accuracy'],
      
      labels={'index': 'epoch', 'value': 'mean squared error'},
      title='Autoencoder Training History (90 filters, balanced train set only)'
   )
   fig.show()

plot_history(history)


In [ ]:
# Function to visualize the encoded images
def visualize_encoded_images(model, dataset, num_images=5):
   # Get a batch of images from the dataset
   for x_batch, _ in dataset.take(1):
      encoded_images = model.layers[0](x_batch)
      decoded_images = model.layers[1](encoded_images)
      break

   # Plot the original and decoded images
   fig, axs = plt.subplots(2, num_images, figsize=(20, 5))
   for i in range(num_images):
      axs[0, i].imshow(x_batch[i])
      axs[0, i].axis('off')
      axs[1, i].imshow(decoded_images[i])
      axs[1, i].axis('off')
      fig.suptitle('Original (top) vs. Decoded (bottom) Images (90 filters, balanced train set only)')

   plt.show()

visualize_encoded_images(conv_ae, test_dataset) 

# Enhanced Pneumonia Detection using Convolutional Autoencoder

In [ ]:
# Build the classifier as a Functional model, including a block of convolutional layers, then the encoder, then a block of dense layers
def build_classifier(encoder):
   # Freeze the encoder layers
   for layer in encoder.layers:
      layer.trainable = False

   # Create an input layer that contains two inputs: the encoder input and a shallow input
   input_= Input((image_size, image_size, 3), name='input')

   # Using encoder as the sole feature extractor
   encoded_features = encoder(input_)
   x = BatchNormalization(name="encoder_batchNorm_1")(encoded_features)
   DNN_output = Dense(1, activation='sigmoid', name='output')(x)

   # Create the model
   model = Model(inputs=input_, outputs=DNN_output)

   return model

# Build the classifier
classifier = build_classifier(conv_encoder)
classifier.compile(
   loss='binary_crossentropy',
   optimizer=tf.keras.optimizers.Nadam(learning_rate=0.05),
   metrics=['accuracy', 'AUC']
)

# SHow the block diagram of the classifier using plot_model
plot_model(classifier, show_shapes=True, show_layer_names=True)


In [ ]:
# Train the classifier
history = classifier.fit(
   balanced_train_generator,
   validation_data=balanced_val_generator,
   epochs=15,
   steps_per_epoch=len(balanced_train_generator),
   validation_steps=len(balanced_val_generator),
   callbacks=[classifier_early_stopping, classifier_model_checkpoint, classifier_reduce_lr]
)

In [ ]:
# Evaluate the classifier
classifier.evaluate(test_generator, steps=len(test_generator))

# Get the true labels
y_true = test_generator.classes
# Get the predicted labels
y_pred = classifier.predict(test_generator)
# Round the predicted labels up and down
y_pred = np.round(y_pred)
# Print the classification report
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Plot the training history using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(y=history.history['auc'], mode='lines', name='AUC'))
fig.update_layout(title=f'Classifier Accuracy (balanced train set only)', xaxis_title='Epochs', yaxis_title='Accuracy')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation'))
fig.update_layout(title='Classifier Loss (balanced train set only)', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()


In [ ]:
# Print the predictions images and labels from the classifier
predictions = np.round(classifier.predict(test_generator, steps=len(test_generator)))
example_true_labels = test_generator.classes
plt.figure(figsize=(10, 10))
# plot the first 6 images in subplots with two columns
for i in range(6):
   plt.subplot(3, 2, i + 1)
   img = next(test_generator)[0][0]
   plt.imshow(img)
   plt.axis('off')
   plt.title(f'Prediction: {predictions[i][0]}, Actual: {example_true_labels[i]}')

# VAE for X-Ray Image Generation

In [ ]:
# Create a VAE model for image generation

class Sampling(tf.keras.layers.Layer): 
   def call(self, inputs):
      mean, log_var = inputs
      return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

codings_size = 90
inputs = Input(shape=[224, 224, 3])
Z = Flatten()(inputs)
Z = Dense(150, activation="relu")(Z)
Z = Dense(100, activation="relu")(Z) 
codings_mean = Dense(codings_size)(Z) # μ 
codings_log_var = Dense(codings_size)(Z) # γ 
codings = Sampling()([codings_mean, codings_log_var]) 

variational_encoder = Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings], name="variational_encoder")

decoder_inputs = Input(shape=[codings_size])
x = Dense(100, activation="relu")(decoder_inputs)
x = Dense(150, activation="relu")(x)
x = Dense(224 * 224 * 3)(x)
outputs = Reshape([224, 224, 3])(x)
variational_decoder = Model(inputs=[decoder_inputs], outputs=[outputs], name="variational_decoder")

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = Model(inputs=[inputs], outputs=[reconstructions], name="variational_ae")

latent_loss = 90 * tf.reduce_sum(1 + codings_log_var - tf.exp(codings_log_var) - tf.square(codings_mean), axis=-1)
print(latent_loss.shape)
variational_ae.add_loss(tf.reduce_mean(latent_loss) / 150528)

variational_ae.compile(
   loss= "mean_squared_error",
   optimizer=tf.keras.optimizers.Nadam(learning_rate=0.1),
   metrics=['accuracy', 'AUC', tf.keras.metrics.MeanSquaredError()]
)

variational_ae.build(input_shape=(None, image_size, image_size, 3))


# Train the variational autoencoder
history = variational_ae.fit(
   train_dataset,
   validation_data=val_dataset,
   epochs=15,
   steps_per_epoch=len(balanced_train_generator),
   validation_steps=len(balanced_val_generator),
   callbacks=[classifier_early_stopping, classifier_model_checkpoint, classifier_reduce_lr]
)

In [ ]:
# Generate images using the VAE
def generate_images(model, num_images=5):
   # Generate random codings
   codings = tf.random.normal(shape=[num_images, codings_size])
   # Decode the codings
   decoded_images = model.layers[1](codings)
   # Plot the decoded images
   fig, axs = plt.subplots(1, num_images, figsize=(20, 5))
   for i in range(num_images):
      axs[i].imshow(decoded_images[i])
      axs[i].axis('off')
      fig.suptitle(f'Generated Images (codings size: {codings_size}, balanced train set only)')

   plt.show()


In [ ]:
# A